# Fichier: LECTURE ET ANALYSE DES CLUSTERS EN STOCK
### Importation des librairies

In [ ]:
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.offline.offline import _plot_html
import plotly.graph_objs as go
print (__version__) # requires version >= 1.9.0
init_notebook_mode(connected=True)

from mpl_toolkits import mplot3d
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
%matplotlib notebook
#%matplotlib inline

from tslearn.clustering import silhouette_score

from utils import series_supp as ss
from utils import data_factory as df
from utils import k_mean as km
from utils import k_shape as ks

### Initialise la factory et créé les instances de Support de Series: ss.SeriesSupp

In [ ]:
cwd = os.getcwd()
os.chdir(cwd)
print(cwd)
factory = df.DataFactory(cwd)
store_path = "cluster\\13_06\\"

RG24 = ss.SeriesSupp(cwd, factory, "RG24")
RG1 = ss.SeriesSupp(cwd, factory, "RG1")
GW = ss.SeriesSupp(cwd, factory, "GW")

### Importation des datasets depuis les fichiers de stockage csv

In [ ]:
GW.import_dataset()
RG24.import_dataset()

### Création des instances de Clustering
Commenter ou décommenter pour choisir le cluster_GW voulu selon Kshape ou Kmean
* GW: Grand Water -> les piezomètres
* RG: Rain gauge -> les pluviomètres

In [ ]:
#cluster_GW = km.Kmean(GW)
cluster_GW = ks.Kshape(GW)
cluster_RG = km.Kmean(RG24)

## Importation du fichier Pickles contenant le clustering voulu
### Veuillez renseigner le nom du fichier ci dessous

In [ ]:
name_file = "kmean_GW_[2013]_allmonths_w_10"

In [ ]:
cluster_GW.read_cluster(store_path, name_file)

## Affichage des cluster
* Prototypes en rouge
* informations en dessous
* repartition des objets dans les cluster

In [ ]:
cluster_GW.ploter.plot_cluster_light()
cluster_GW.show_info()
cluster_GW.cluster_counter()
cluster_GW.counter

#### Silhouette score cluster quality indice
could be huge and long computation due to massive dataset

In [ ]:
#sil = silhouette_score(cluster_GW.ts, cluster_GW.ts_clust, metric="softdtw", metric_params = {"gamma_sdtw": .01})
#print("Silhouette score: " + str(sil))

In [ ]:
cluster_GW.capteur_parser()
cluster_GW.ploter.change_mode(3)
#cluster_GW.cluster_by_fullname

## Analyse: n_obs est le numero du cluster à observer

In [ ]:
n_obs = 6
cluster_GW.geo.plotly_3D(cluster_GW.cluster_by_name[n_obs])
cluster_GW.geo.distance_matrix()
cluster_GW.ploter.plot_histo_2(n_obs)
#cluster_GW.clust_hoverview(n_obs)

# Affichage des données GW a parametrer
#cluster_GW.ploter.change_mode(3)
#cluster_RG.ploter.plot_scatter(cluster_RG.ss.dataset)

In [ ]:
cluster_GW.nb_week

In [ ]:
#cluster_RG.ss.reset_dataset()
#cluster_RG.ss.years = [2013, 2014, 2015]

In [ ]:
#cluster_RG.ss.split_data_years()
#cluster_RG.ss.split_data_months()
#cluster_RG.ss.split_data_weeks()

In [ ]:
#cluster_RG = km.Kmean(RG24)
#cluster_RG.ploter.plot_scatter_by_capteur(cluster_RG.ss.dataset, ["24h_RG007", "24h_RG017"])

In [ ]:
#for k, v in cluster_GW.ss.dataset.items():
#    print(k)

## Recherche et test sur le SAX

In [ ]:
from prefixspan import PrefixSpan
from utils import sax_engine as sq
from utils import prefix_span as ps

In [ ]:
se = sq.SaxEngine(8, 50)

In [ ]:
se.reset()

In [ ]:
se.set_nb_symbol(5)
se.set_nb_segment(40)

In [ ]:
all_ts = cluster_GW.get_cluster_n(n_obs)

In [ ]:
se.fit(all_ts)
se.run()

In [ ]:
psm = ps.PrefixSpanManager(se)

In [ ]:
psm.run()

In [ ]:
psm.topk(5)

In [ ]:
n = 25

In [ ]:
traw = psm.se_instance.raw_data[n]
traw.shape

In [ ]:
tinv = psm.se_instance.sax_data_inv[n]
tstr = psm.se_instance.sax_data[n]
tinv.shape

In [ ]:
#tinv

In [ ]:
#len(psm.se_instance.sax_data_inv)
#len(psm.se_instance.sax_data)
#len(psm.se_instance.raw_data)
#len(tinv.ravel())
#len(tstr.ravel())
#len(traw)

In [ ]:
plt.plot(traw.ravel(), "b-", alpha=0.4)
plt.plot(tinv.ravel(), "b-")
plt.plot(tstr.ravel(), "r-")